In [53]:
!pip install xarray==0.21.1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 25.9 MB/s eta 0:00:00
  Created wheel for asciitree: filename=asciitree-0.3.3-py3-none-any.whl size=5033 sha256=9a52e8c84436f2a02151395980e3bf4d984dab5e2e22c6e4e8ec9bdfd0920b9f
  Stored in directory: /Users/ayalahlou/Library/Caches/pip/wheels/71/c1/da/23077eb3b87d24d6f3852ed1ed1a1ac2d3c885ad6ebd2b4a07
Successfully built asciitree
  Attempting uninstall: xarray
    Found existing installation: xarray 0.21.1
    Uninstalling xarray-0.21.1:
      Successfully uninstalled xarray-0.21.1


In [43]:
!pip install dask

  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached toolz-1.0.0-py3-none-any.whl.metadata (5.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.0 MB/s eta 0:00:00
Using cached click-8.1.7-py3-none-any.whl (97 kB)
Using cached toolz-1.0.0-py3-none-any.whl (56 kB)


In [1]:
import argparse
import os
from netCDF4 import Dataset
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import xesmf as xe
import dask
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [ ]:
def get_csif(year,lat_s, lat_e, lon_s, lon_e, pft_path, pft_code):
    """
    Retrieve and process CSIF (Chlorophyll Fluorescence) data for a specified year and geographical region, 
    and apply a mask based on Plant Functional Type (PFT).
    
    Parameters:
        year (int): The year for which to retrieve the CSIF data.
        lat_s (float): The starting latitude of the region of interest.
        lat_e (float): The ending latitude of the region of interest.
        lon_s (float): The starting longitude of the region of interest.
        lon_e (float): The ending longitude of the region of interest.
        pft_path (str): The file path to the PFT map.
        pft_code (str): The code representing the Plant Functional Type of interest.
    
    Returns:
        xarray.Dataset: The processed CSIF data with the applied PFT mask and interpolated time series.
    """
    
    
    pft_dictionnary = {
    "TrBE": 1, #Tropical Broadleaf Evergreen # 
    "TrBD": 2, #Tropical Broadleaf Deciduous #
    "TeBE": 3, #Temperate Broadleaf Evergreen #
    "TeNE": 4, #Temperate Needleleaf Evergreen #
    "TeBD": 5, #Temperate Broadleaf Deciduous #
    "BoNE": 6, #Boreal Needleleaf Evergreen #
    "BoND": 7, #Boreal Needleleaf Deciduous #
    "Sav": 8, #Savanna #
    "Gra": 9, #Grassland #
    "Sch": 10, #Schrub #
    "Tun": 11, #Tundra #
    "Des": 12, #Desert
    "Ice": 13, #Polar-Desert_or_Rock_or_Ice 
    "Wat": 14, #Water
    "Cro": 15, #Cropland
    "Pas": 16, #Pastureland 
    "Urb": 17, #Urban 
    }

    desired_lat_values = np.linspace(90.0, -89.75, 720)
    desired_lon_values = np.arange(-180.0, 180, 0.25)

    csif_path='/Users/ayalahlou/Desktop/All Data/csif_biweekly_lowres/'+str(year)+'_csif_24x720x1440.nc'
    csif = xr.open_dataset(csif_path)
    csif['latitude']=desired_lat_values
    csif['longitude']=desired_lon_values

    csif = csif.sel(latitude=slice(lat_e, lat_s))
    csif = csif.sel(longitude=slice(lon_s, lon_e))


    #for jan1st to Jan15th
    if year>1982:
        csif_path_2='/Users/ayalahlou/Desktop/All Data/csif_biweekly_lowres/'+str(year-1)+'_csif_24x720x1440.nc'
        csif_2 = xr.open_dataset(csif_path_2)
        csif_2['latitude']=desired_lat_values
        csif_2['longitude']=desired_lon_values
        #slice according to the area of interest
        csif_2 = csif_2.sel(latitude=slice(lat_e, lat_s))
        csif_2 = csif_2.sel(longitude=slice(lon_s, lon_e))
        csif_2 = csif_2.sel(time=slice(str(year-1)+'-12-30', str(year)+'-01-01'))
        ds = xr.merge([csif_2, csif], join='outer') 
    elif year==1982:
        ds=csif

    sif_inst= ds['sif_clear_inst'].values


    sif_inst=sif_inst.reshape(len(ds['time']),len(csif['latitude']), len(csif['longitude']))

    #flip upside down each netcdf for each month
    #for i in range(len(ds['time'])):
        #sif_inst[i]=np.flipud(sif_inst[i])

    ds= ds.drop_vars(["sif_clear_daily"])

    #OPEN binary pft map
    pft_path = pft_path
    pft = xr.open_dataset(pft_path)
    pft = pft.sel(latitude=slice(lat_e, lat_s))
    pft = pft.sel(longitude=slice(lon_s, lon_e))
    arr_pft= pft['Dominant_type'].values           #define PFT of interest 
    #arr_pft[0]=np.flipud(arr_pft[0])
    
    #apply mask to csif data
    mask = (arr_pft[0] != pft_dictionnary[pft_code]) #mask of everything but pft of interest
    sif_inst[:, mask] = np.nan # Apply the mask to sif_inst for all 24 slices


    start_date = pd.to_datetime(ds['time'].min().values)
    end_date = pd.to_datetime(ds['time'].max().values)
    new_time_array = pd.date_range(start=start_date, end=end_date, freq='D')

    ds_interpolated = ds.interp(time=new_time_array, method='linear')
    ds_interpolated['sif_clear_inst'] = ds_interpolated['sif_clear_inst'].rolling(time=10, min_periods=1, center=True).mean()

    if year>1982:
        ds_interpolated=ds_interpolated.isel(time=slice(1,None))
        
    return ds_interpolated


def get_era5(year, lat_s, lat_e, lon_s, lon_e, precp_path,  max_temp_path, min_temp_path,rad_path, sm_path):
    """
    Retrieve and combine ERA5 climate data for a specified year and geographical region.
    Parameters:
    year (int): The year for which to retrieve the data.
    lat_s (float): The starting latitude of the region.
    lat_e (float): The ending latitude of the region.
    lon_s (float): The starting longitude of the region.
    lon_e (float): The ending longitude of the region.
    precp_path (str): The file path to the precipitation data directory.
    max_temp_path (str): The file path to the maximum temperature data directory.
    min_temp_path (str): The file path to the minimum temperature data directory.
    rad_path (str): The file path to the radiation data directory.
    sm_path (str): The file path to the soil moisture data directory.
    Returns:
    xarray.Dataset: A combined dataset containing minimum temperature, maximum temperature, radiation, precipitation, and soil moisture data for the specified year and region.
    """
    

    #_________________________________________RADIATION___________________________________
    print('getting radiation for', str(year))                                                                       
    year_folder_path = os.path.join(rad_path, str(year))                                                                         
    ds_rad = read_netcdf_file(year_folder_path, lat_s, lat_e, lon_s, lon_e,)

    #_________________________________________PRECIPITATION_________________________________________
    print('getting precipitation for', str(year))                                                                       
    year_folder_path = os.path.join(precp_path, str(year))                                                                         
    ds_precp = read_netcdf_file(year_folder_path, lat_s, lat_e, lon_s, lon_e,)

    #_________________________________________MAX_TEMP________________________________________
    print('getting max temp for', str(year))                                                                       
    year_folder_path = os.path.join(max_temp_path, str(year))                                                                         
    ds_max_temp = read_netcdf_file(year_folder_path, lat_s, lat_e, lon_s, lon_e,)

    #_________________________________________MIN_TEMP________________________________________
    print('getting min temp for', str(year))                                                                       
    year_folder_path = os.path.join(min_temp_path, str(year))                                                                         
    ds_min_temp = read_netcdf_file(year_folder_path, lat_s, lat_e, lon_s, lon_e,)
    
    #_________________________________________SM________________________________________
    print('getting soil moisture for', str(year))                                                                       
    year_folder_path = sm_path+"/era5_SM_layer1_"+str(year)+".nc"                                                                         
    ds_sm = read_netcdf_file(year_folder_path, lat_s, lat_e, lon_s, lon_e, True)
    
    combined_ds = xr.merge([ds_min_temp, ds_max_temp, ds_rad, ds_precp, ds_sm])
    combined_ds = combined_ds.rename({'time': 'time', 'lat': 'latitude', 'lon': 'longitude','air_temperature_at_2_metres_1hour_Minimum': 'tmin', 'air_temperature_at_2_metres_1hour_Maximum': 'tmax', 'integral_wrt_time_of_surface_direct_downwelling_shortwave_flux_in_air_1hour_Accumulation': 'radiation', 'precipitation_amount_1hour_Accumulation':'precipitation' ,"swvl1":"sm" })
    return combined_ds

def get_1982_ref(lat_s, lat_e, lon_s, lon_e, pft_path, precp_path,  max_temp_path, min_temp_path,rad_path, sm_path, pft_code):
    """
    Retrieve and combine CSIF and ERA5 datasets for the year 1982 within specified latitude and longitude bounds.
    Parameters:
    lat_s (float): Starting latitude.
    lat_e (float): Ending latitude.
    lon_s (float): Starting longitude.
    lon_e (float): Ending longitude.
    pft_path (str): File path to the PFT (Plant Functional Type) data.
    precp_path (str): File path to the precipitation data.
    max_temp_path (str): File path to the maximum temperature data.
    min_temp_path (str): File path to the minimum temperature data.
    rad_path (str): File path to the radiation data.
    sm_path (str): File path to the soil moisture data.
    pft_code (int): PFT code to filter the data.
    Returns:
    xarray.Dataset: Combined dataset containing CSIF and ERA5 data for the year 1982.
    """
    
    year=1982
    csif_ds= get_csif(year,lat_s, lat_e, lon_s, lon_e, pft_path, pft_code)
    era5_ds= get_era5(year,lat_s, lat_e, lon_s, lon_e, precp_path,  max_temp_path, min_temp_path,rad_path, sm_path)
    # We don't have CSIF data for before 1982-01-15 so we truncate the era5 data for compatibility
    era5_ds = era5_ds.sel(time=slice('1982-01-15',None))
    combined_ds = xr.merge([csif_ds, era5_ds])
    return combined_ds

def read_netcdf_file(filepath,lat_s, lat_e, lon_s, lon_e, SM=False):
    """
    Read a netCDF file and extract data within specified latitude and longitude bounds.
    Parameters:
    filepath (str): File path to the netCDF file.
    lat_s (float): Starting latitude.
    lat_e (float): Ending latitude.
    lon_s (float): Starting longitude.
    lon_e (float): Ending longitude.
    SM (bool): Whether the data is soil moisture data.
    Returns:
    xarray.Dataset: Dataset containing the extracted data.
    """
    if SM:
        ds = xr.open_dataset(filepath)
        ds['longitude'] = (ds['longitude'] + 180) % 360 - 180
        ds = xr.concat([ds.sel(longitude=slice(-180, -0.25)), ds.sel(longitude=slice(0, 179.75))], dim="longitude") #center around africa
        ds = ds.sel(latitude=slice(lat_e, lat_s))
        ds = ds.sel(longitude=slice(lon_s, lon_e))
        ds=ds.rename({'latitude':'lat', 'longitude':'lon'})
    else:
        ds = xr.open_mfdataset(filepath+'/*.nc')
        ds['lon'] = (ds['lon'] + 180) % 360 - 180
        ds = xr.concat([ds.sel(lon=slice(-180, -0.25)), ds.sel(lon=slice(0, 179.75))], dim="lon") #center around africa
        ds = ds.sel(lat=slice(lat_e, lat_s))
        ds = ds.sel(lon=slice(lon_s, lon_e))
        ds=ds.rename({'time1':'time'})

    return ds

def process_one_year(year,lat_s, lat_e, lon_s, lon_e, pft_path, precp_path,  max_temp_path, min_temp_path,rad_path, sm_path, ref_1982, pft_code, output_path):
    """
    Process and save data for a single year.
    Parameters:
    year (int): The year to process.
    lat_s (float): The starting latitude of the region of interest.
    lat_e (float): The ending latitude of the region of interest.
    lon_s (float): The starting longitude of the region of interest.
    lon_e (float): The ending longitude of the region of interest.
    pft_path (str): The file path to the PFT map.
    precp_path (str): The file path to the precipitation data.
    max_temp_path (str): The file path to the maximum temperature data.
    min_temp_path (str): The file path to the minimum temperature data.
    rad_path (str): The file path to the radiation data.
    sm_path (str): The file path to the soil moisture data.
    ref_1982 (xarray.Dataset): The reference dataset for the year 1982.
    pft_code (str): The code representing the Plant Functional Type of interest.
    output_path (str): The output directory to save the processed data.
    """
    if year==1982:
        concatenated = ref_1982.stack(location=('latitude', 'longitude'))
        concatenated = concatenated.dropna(dim='location', how='all', subset=['sif_clear_inst'] )
        concatenated = concatenated.reset_index('location')
        concatenated['sif_clear_inst'] = concatenated['sif_clear_inst'].interpolate_na(dim='time', method="cubic", fill_value="extrapolate")
        print('year '+str(year)+' before dropping for any nan'+str( len(concatenated['location'])))
        concatenated = concatenated.dropna(dim='location', how='any', subset=['sif_clear_inst'] )
        result_ds = concatenated.to_dataframe().reset_index()
        print(year, len(result_ds['location']), len(result_ds['time']))
        result_ds.to_parquet(output_path+'/full_'+str(year)+'_daily_NoNAN.parquet', compression='snappy', engine='pyarrow')
    else:
        #get csif for current year 
        csif_ds= get_csif(year,lat_s, lat_e, lon_s, lon_e, pft_path, pft_code)
        #get era 5 for current year
        era5_ds= get_era5(year,lat_s, lat_e, lon_s, lon_e, precp_path,  max_temp_path, min_temp_path,rad_path, sm_path)
        #merge 
        combined_ds = xr.merge([csif_ds, era5_ds])
        #concatenate xarray dataset with 1982 dataset 
        concatenated = xr.concat([ref_1982,combined_ds], dim='time')
        #stack 
        concatenated = concatenated.stack(location=('latitude', 'longitude'))
        #remove na 
        concatenated = concatenated.dropna(dim='location', how='all', subset=['sif_clear_inst'] )
        if year>=2001:
            current_year = concatenated.sel(time=slice(str(year)+'-01-01',None))#['sif_clear_inst'] = concatenated.sel(time=slice(None,str(year)+'-01-01'))['sif_clear_inst'].interpolate_na(dim='time', method="cubic", fill_value="extrapolate", max_gap='60D')
            current_year['sif_clear_inst'] = current_year['sif_clear_inst'].interpolate_na(dim='time', method="cubic", fill_value="extrapolate")
            current_year = current_year.fillna(0)
            again_together = xr.concat([concatenated.sel(time=slice(None, '1982-12-31')),current_year], dim='time')
            concatenated = again_together.dropna(dim='location', thresh=367, subset=['sif_clear_inst'])
        concatenated = concatenated.reset_index('location')
        concatenated['sif_clear_inst'] = concatenated['sif_clear_inst'].interpolate_na(dim='time', method="cubic", fill_value="extrapolate")
        print('year '+str(year)+' before dropping for any nan'+str( len(concatenated['location'])))
        concatenated = concatenated.dropna(dim='location', how='any', subset=['sif_clear_inst'] )
        #remove 1982 data 
        result_ds=concatenated.sel(time=slice(str(year)+'-01-01',None))
        print(year, len(result_ds['location']), len(result_ds['time']))
        #save 
        result_ds = result_ds.to_dataframe().reset_index()
        result_ds.to_parquet(output_path+'full_'+str(year)+'_daily_NoNAN.parquet', compression='snappy', engine='pyarrow')
        
        
def plot_map(ds_variable,title=None):
    """
    Plot a map of a variable in an xarray dataset.
    Parameters:
    ds_variable (xarray.DataArray): The variable to plot.
    title (str): The title of the plot.
    """

    for i in range(24):
        ds_variable[i]=np.flipud(ds_variable[i])
    
    # Assuming sif_inst has shape (24, 141, 1440)
    # Select a specific time slice to plot, e.g., the first time slice
    time_slice = 0
    data_to_plot = ds_variable[time_slice]
    # Assuming you have latitude and longitude arrays
    # Replace these with your actual latitude and longitude arrays
    lat = np.linspace(30, 75, data_to_plot.shape[0])
    lon = np.linspace(-180, 180, data_to_plot.shape[1])

    # Create a figure and axis with a map projection
    fig, ax = plt.subplots(figsize=(15, 4), subplot_kw={'projection': ccrs.PlateCarree()})

    # Plot the data
    c = ax.pcolormesh(lon, lat, data_to_plot, transform=ccrs.PlateCarree(), cmap='viridis')

    # Add coastlines and other features
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.add_feature(cfeature.LAND, edgecolor='black')
    ax.add_feature(cfeature.OCEAN)


    # Add a colorbar
    fig.colorbar(c, ax=ax, orientation='vertical', label='SIF')

    # Set the title
    ax.set_title(title)

    # Show the plot
    plt.show()


In [3]:
year=2001

lat_e=75
lat_s=30
lon_s=-180
lon_e=180

pft_code= 'BoNE' #climate_leaftype_seasonality
csif_path='/Users/ayalahlou/Desktop/All Data/csif_biweekly_lowres/'+str(year)+'_csif_24x720x1440.nc'
pft_path = "/Users/ayalahlou/Downloads/regridded_landcover-HYDEAREAVEG-fc_Historical_Land-Cover_Change_and_Land-Use_Conversions_Global_Dataset__HYDE_AREAVEG_Feature_Collection_best.nc"#2011
precp_path='/Users/ayalahlou/Desktop/All Data/era5_daily/precip'
max_temp_path='/Users/ayalahlou/Desktop/All Data/era5_daily/max_temp'
min_temp_path='/Users/ayalahlou/Desktop/All Data/era5_daily/min_temp'
rad_path='/Users/ayalahlou/Desktop/All Data/era5_daily/radiation'
sm_path= "/Users/ayalahlou/Desktop/All Data/scripts/soilmoisture"

output_path='/Users/ayalahlou/Desktop/All Data/Boreal_Clustering/'+pft_code+"/"


In [11]:
ref_1982= get_1982_ref(lat_s, lat_e, lon_s, lon_e, pft_path, precp_path,  max_temp_path, min_temp_path,rad_path, sm_path, pft_code)
for year in range(1982,2022):
    process_one_year(year,lat_s, lat_e, lon_s, lon_e, pft_path, precp_path,  max_temp_path, min_temp_path, rad_path, sm_path, ref_1982, pft_code, output_path)


getting radiation for 1982
getting precipitation for 1982
getting max temp for 1982
getting min temp for 1982
getting soil moisture for 1982
year 1982 before dropping for any nan11502
1982 4037202 4037202
getting radiation for 1983
getting precipitation for 1983
getting max temp for 1983
getting min temp for 1983
getting soil moisture for 1983
year 1983 before dropping for any nan11502
1983 11502 365
getting radiation for 1984
getting precipitation for 1984
getting max temp for 1984
getting min temp for 1984
getting soil moisture for 1984
year 1984 before dropping for any nan11502
1984 11502 366
getting radiation for 1985
getting precipitation for 1985
getting max temp for 1985
getting min temp for 1985
getting soil moisture for 1985
year 1985 before dropping for any nan11502
1985 11502 365
getting radiation for 1986
getting precipitation for 1986
getting max temp for 1986
getting min temp for 1986
getting soil moisture for 1986
year 1986 before dropping for any nan11502
1986 11502 365

In [18]:
csif_path='/Users/ayalahlou/Desktop/All Data/csif_biweekly_lowres/'+str(year)+'_csif_24x720x1440.nc'
pft_path = "/Users/ayalahlou/Downloads/regridded_landcover-HYDEAREAVEG-fc_Historical_Land-Cover_Change_and_Land-Use_Conversions_Global_Dataset__HYDE_AREAVEG_Feature_Collection_best.nc"#2011
precp_path='/Users/ayalahlou/Desktop/All Data/era5_daily/precip'
max_temp_path='/Users/ayalahlou/Desktop/All Data/era5_daily/max_temp'
min_temp_path='/Users/ayalahlou/Desktop/All Data/era5_daily/min_temp'
rad_path='/Users/ayalahlou/Desktop/All Data/era5_daily/radiation'
sm_path= "/Users/ayalahlou/Desktop/All Data/scripts/soilmoisture"

# 
lat_e=70
lat_s=-60
lon_s=-240
lon_e=180

selected_pfts=['TrBE', 'TrBD', 'TeBE', 'TeNE', 'TeBD', 'BoND', 'Sav', 'Gra', 'Sch', 'Tun']
for pft_code in selected_pfts:
    if pft_code=="Tun":
        lat_e=90
        lat_s=60
        lon_s=-180
        lon_e=180
        
    #make output directory
    output_path='/Users/ayalahlou/Desktop/All Data/Boreal_Clustering/'+pft_code+"/"
    if not os.path.exists(output_path):
        os.makedirs(output_path)
        
    print('__________processing______________', pft_code)
    ref_1982= get_1982_ref(lat_s, lat_e, lon_s, lon_e, pft_path, precp_path,  max_temp_path, min_temp_path,rad_path, sm_path, pft_code)
    for year in range(1982,2022):
        process_one_year(year,lat_s, lat_e, lon_s, lon_e, pft_path, precp_path,  max_temp_path, min_temp_path, rad_path, sm_path, ref_1982, pft_code, output_path)

        
    
    

__________processing______________ TrBE
getting radiation for 1982
getting precipitation for 1982
getting max temp for 1982
getting min temp for 1982
getting soil moisture for 1982
year 1982 before dropping for any nan13674
1982 4799574 4799574
getting radiation for 1983
getting precipitation for 1983
getting max temp for 1983
getting min temp for 1983
getting soil moisture for 1983
year 1983 before dropping for any nan13674
1983 13674 365
getting radiation for 1984
getting precipitation for 1984
getting max temp for 1984
getting min temp for 1984
getting soil moisture for 1984
year 1984 before dropping for any nan13674
1984 13674 366
getting radiation for 1985
getting precipitation for 1985
getting max temp for 1985
getting min temp for 1985
getting soil moisture for 1985
year 1985 before dropping for any nan13674
1985 13674 365
getting radiation for 1986
getting precipitation for 1986
getting max temp for 1986
getting min temp for 1986
getting soil moisture for 1986
year 1986 before 